# New Section

In [22]:
import datetime
print(f"Notebook last run (end-to-end): {datetime.datetime }")
# # Load the TensorBoard notebook extension
# %load_ext tensorboard
import tensorflow as tf
import os

print(f"TensorFlow version: {tf.__version__}")

Notebook last run (end-to-end): <class 'datetime.datetime'>
TensorFlow version: 2.15.0


In [23]:
def walk_through_dir(dir_path):
  """
  Walks through dir_path returning its contents.

  Args:
    dir_path (str): target directory

  Returns:
    A print out of:
      number of subdiretories in dir_path
      number of images (files) in each subdirectory
      name of each subdirectory
  """
  for dirpath, dirnames, filenames in os.walk(dir_path):
    print(f"There are {len(dirnames)} directories and {len(filenames)} images in '{dirpath}'.")

In [26]:
walk_through_dir("/content/plant_dataset.rar")

In [30]:
import zipfile

def unzip_data(filename, extract_to="."):
    """
    Unzips a file into the specified directory.

    Args:
        filename (str): The filepath of the ZIP file to be unzipped.
        extract_to (str): The directory where the contents will be extracted.
                          Defaults to the current directory.
    """
    with zipfile.ZipFile(filename, 'r') as zip_ref:
        zip_ref.extractall(extract_to)



In [32]:

# Example usage:
zip_file = "plant_dataset.zip"
unzip_data(zip_file, "extracted_data")  # Extracts to the "extracted_data" directory

In [33]:
#from helper_functions import create_tensorboard_callback, plot_loss_curves, unzip_data, walk_through_dir
walk_through_dir("/content/extracted_data/plant_dataset")
# Create training and test directories
train_dir = "/content/extracted_data/plant_dataset/train"
test_dir = "/content/extracted_data/plant_dataset/test"

There are 2 directories and 0 images in '/content/extracted_data/plant_dataset'.
There are 3 directories and 0 images in '/content/extracted_data/plant_dataset/train'.
There are 0 directories and 149 images in '/content/extracted_data/plant_dataset/train/guava'.
There are 0 directories and 150 images in '/content/extracted_data/plant_dataset/train/mango'.
There are 0 directories and 150 images in '/content/extracted_data/plant_dataset/train/tulsi'.
There are 3 directories and 0 images in '/content/extracted_data/plant_dataset/test'.
There are 0 directories and 30 images in '/content/extracted_data/plant_dataset/test/guava'.
There are 0 directories and 30 images in '/content/extracted_data/plant_dataset/test/mango'.
There are 0 directories and 33 images in '/content/extracted_data/plant_dataset/test/tulsi'.


In [34]:
import tensorflow as tf
IMG_SIZE = (224,224)
train_data_leaves = tf.keras.preprocessing.image_dataset_from_directory(directory =  train_dir,
                                                                            image_size = IMG_SIZE,
                                                                            label_mode = "categorical",
                                                                            batch_size = 32)
test_data_leaves = tf.keras.preprocessing.image_dataset_from_directory(directory=test_dir,
                                                                           image_size=IMG_SIZE,
                                                                           label_mode="categorical")
train_data_leaves
train_data_leaves.class_names
for images, labels in train_data_leaves.take(1):
  print(images, labels)
train_data_leaves.take(1)

Found 449 files belonging to 3 classes.
Found 93 files belonging to 3 classes.
tf.Tensor(
[[[[255.       255.       255.      ]
   [255.       255.       255.      ]
   [255.       255.       255.      ]
   ...
   [249.74745  249.74745  249.74745 ]
   [250.67857  250.67857  250.67857 ]
   [250.78572  250.78572  250.78572 ]]

  [[255.       255.       255.      ]
   [254.93112  254.93112  254.93112 ]
   [254.35715  254.35715  254.35715 ]
   ...
   [247.5357   247.5357   247.5357  ]
   [249.18112  249.18112  249.18112 ]
   [249.97704  249.97704  249.97704 ]]

  [[254.96428  254.96428  254.96428 ]
   [253.99234  253.99234  253.99234 ]
   [253.76276  253.76276  253.76276 ]
   ...
   [245.72704  245.72704  245.72704 ]
   [247.79337  247.79337  247.79337 ]
   [248.96428  248.96428  248.96428 ]]

  ...

  [[248.75003  246.75003  247.75003 ]
   [245.4286   243.6429   244.75003 ]
   [242.23982  242.23982  244.23982 ]
   ...
   [225.08163  223.08163  226.08163 ]
   [224.2143   222.2143   225.214

<_TakeDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 3), dtype=tf.float32, name=None))>

In [35]:

# 1. Create base model with tf.keras.applications
base_model = tf.keras.applications.efficientnet_v2.EfficientNetV2B0(include_top=False)

# OLD
# base_model = tf.keras.applications.EfficientNetB0(include_top=False)

# 2. Freeze the base model (so the pre-learned patterns remain)
base_model.trainable = False

24274472/24274472 [==============================] - 0s 0us/step


In [36]:

def create_tensorboard_callback(dir_name, experiment_name):
  """
  Creates a TensorBoard callback instand to store log files.

  Stores log files with the filepath:
    "dir_name/experiment_name/current_datetime/"

  Args:
    dir_name: target directory to store TensorBoard log files
    experiment_name: name of experiment directory (e.g. efficientnet_model_1)
  """
  log_dir = dir_name + "/" + experiment_name + "/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
  tensorboard_callback = tf.keras.callbacks.TensorBoard(
      log_dir=log_dir
  )
  print(f"Saving TensorBoard log files to: {log_dir}")
  return tensorboard_callback


In [37]:

# 3. Create inputs into the base model
inputs = tf.keras.layers.Input(shape=(224, 224, 3), name="input_layer")

# 4. If using ResNet50V2, add this to speed up convergence, remove for EfficientNetV2
# x = tf.keras.layers.experimental.preprocessing.Rescaling(1./255)(inputs)

# 5. Pass the inputs to the base_model (note: using tf.keras.applications, EfficientNetV2 inputs don't have to be normalized)
x = base_model(inputs)
# Check data shape after passing it to base_model
print(f"Shape after base_model: {x.shape}")

# 6. Average pool the outputs of the base model (aggregate all the most important information, reduce number of computations)
x = tf.keras.layers.GlobalAveragePooling2D(name="global_average_pooling_layer")(x)
print(f"After GlobalAveragePooling2D(): {x.shape}")

# 7. Create the output activation layer
outputs = tf.keras.layers.Dense(3, activation="softmax", name="output_layer")(x)

# 8. Combine the inputs with the outputs into a model
model_0 = tf.keras.Model(inputs, outputs)

# 9. Compile the model
model_0.compile(loss='categorical_crossentropy',
              optimizer=tf.keras.optimizers.Adam(),
              metrics=["accuracy"])

# 10. Fit the model (we use less steps for validation so it's faster)
history = model_0.fit(train_data_leaves,
                                 epochs=5,
                                 steps_per_epoch=len(train_data_leaves),
                                 validation_data=train_data_leaves,
                                 # Go through less of the validation data so epochs are faster (we want faster experiments!)
                                 validation_steps=int(0.25 * len(train_data_leaves)),
                                 # Track our model's training logs for visualization later
                                 callbacks=[create_tensorboard_callback("transfer_learning", "10_percent_feature_extract")])

Shape after base_model: (None, 7, 7, 1280)
After GlobalAveragePooling2D(): (None, 1280)
Saving TensorBoard log files to: transfer_learning/10_percent_feature_extract/20240319-175408
Epoch 1/5
15/15 [==============================] - 47s 2s/step - loss: 0.6177 - accuracy: 0.8174 - val_loss: 0.2372 - val_accuracy: 1.0000
Epoch 2/5
15/15 [==============================] - 30s 2s/step - loss: 0.1609 - accuracy: 0.9978 - val_loss: 0.0891 - val_accuracy: 1.0000
Epoch 3/5
15/15 [==============================] - 32s 2s/step - loss: 0.0724 - accuracy: 1.0000 - val_loss: 0.0512 - val_accuracy: 1.0000
Epoch 4/5
15/15 [==============================] - 30s 2s/step - loss: 0.0454 - accuracy: 1.0000 - val_loss: 0.0333 - val_accuracy: 1.0000
Epoch 5/5
15/15 [==============================] - 34s 2s/step - loss: 0.0327 - accuracy: 1.0000 - val_loss: 0.0261 - val_accuracy: 1.0000


In [39]:
converter = tf.lite.TFLiteConverter.from_keras_model(model_0)
tflite_model = converter.convert()

with open("model.tflite", 'wb') as f:
  f.write(tflite_model)

In [41]:
model_0.evaluate(test_data_leaves)

3/3 [==============================] - 5s 2s/step - loss: 0.0831 - accuracy: 1.0000


[0.08309602737426758, 1.0]